In [0]:
# Use the BLOB endpoint (wasbs) instead of ADLS dfs
spark.conf.set(
    "fs.azure.account.key.capstoneg7.blob.core.windows.net",
    "EMwjEmmSadGVl1SGKR2zmccXiP2If5QGN7M6MdI3Ng/WDisQLrsv+LxnHbQ6sFJVpUR175vJDAEf+ASt1xgtpQ=="
)

# Sanity check
display(dbutils.fs.ls("wasbs://environment@capstoneg7.blob.core.windows.net/"))
display(dbutils.fs.ls("wasbs://environment@capstoneg7.blob.core.windows.net/bronze/"))

  


path,name,size,modificationTime
wasbs://environment@capstoneg7.blob.core.windows.net/bronze/,bronze/,0,0


path,name,size,modificationTime
wasbs://environment@capstoneg7.blob.core.windows.net/bronze/city_day.csv,city_day.csv,2574056,1762840784000


In [0]:
from pyspark.sql import functions as F

storage = "capstoneg7"
bronze = f"wasbs://environment@{storage}.blob.core.windows.net/bronze/city_day.csv"
silver = f"wasbs://environment@{storage}.blob.core.windows.net/silver/air_quality_clean"

df = (spark.read.option("header", True).option("inferSchema", True).csv(bronze))

# (same cleaning as before)
for c in df.columns:
    df = df.withColumnRenamed(c, c.strip())

if "PM2.5" in df.columns:
    df = df.withColumnRenamed("PM2.5", "pm25")
elif "pm25" not in df.columns:
    display(df.limit(5))
    raise Exception("PM2.5 column not found. Adjust the name.")

if "Date" in df.columns:
    df = df.withColumn("date", F.to_date("Date"))
elif "date" in df.columns:
    df = df.withColumn("date", F.to_date("date"))
else:
    display(df.limit(5))
    raise Exception("Date column not found. Adjust the name.")

df = df.dropDuplicates().filter(F.col("pm25") < 1000)
mean_pm25 = df.agg(F.avg("pm25")).first()[0]
df = df.na.fill({"pm25": float(mean_pm25)})
df = df.withColumn("year", F.year("date")).withColumn("month", F.month("date"))

(df.write.format("delta").mode("overwrite").partitionBy("year","month").save(silver))
display(df.limit(10))


City,date,pm25,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket,year,month
Ahmedabad,2015-02-17,80.61,null,15.96,21.04,35.67,null,15.96,54.7,36.2,8.14,18.75,2.52,479.0,Severe,2015,2
Ahmedabad,2015-03-04,93.88,null,18.21,29.38,47.01,null,18.21,62.75,46.79,5.21,2.55,1.55,383.0,Very Poor,2015,3
Ahmedabad,2015-08-29,39.86,null,16.98,14.83,31.81,null,16.98,10.02,96.83,0.0,0.01,0.02,301.0,Very Poor,2015,8
Ahmedabad,2016-09-11,18.04,null,7.3,7.02,17.19,null,7.3,15.49,19.0,0.61,2.32,0.36,208.0,Poor,2016,9
Ahmedabad,2016-10-30,182.59,null,26.06,18.2,44.25,null,26.06,23.81,41.0,8.18,19.9,10.1,457.0,Severe,2016,10
Ahmedabad,2018-01-02,76.51,null,9.26,76.69,44.32,null,9.26,103.46,93.51,14.29,61.83,7.48,256.0,Poor,2018,1
Ahmedabad,2018-03-16,80.51,null,80.75,83.56,104.64,null,80.75,45.11,36.86,19.32,76.59,13.9,1274.0,Severe,2018,3
Ahmedabad,2019-05-21,24.6,103.88,11.03,81.24,52.21,null,11.03,80.74,46.65,15.31,82.95,11.66,321.0,Very Poor,2019,5
Ahmedabad,2019-08-21,36.44,151.31,24.36,60.1,54.16,null,24.36,40.9,69.33,2.03,30.38,0.49,465.0,Severe,2019,8
Ahmedabad,2019-09-25,61.46,103.64,23.56,112.12,81.56,null,23.56,81.76,60.38,1.15,23.41,2.04,572.0,Severe,2019,9


In [0]:
silver = "wasbs://environment@capstoneg7.blob.core.windows.net/silver/air_quality_clean"
gold   = "wasbs://environment@capstoneg7.blob.core.windows.net/gold/air_quality_monthly"


In [0]:
USE_WASBS = True  # set to False if you are on abfss/dfs
